In [3]:
#%pip install pandas astropy numpy seaborn pandarallel keras ray scikit-learn tensorflow astroNN

In [1]:
import pandas as pd
import glob
import astropy
from astropy.io import fits
import numpy as np
import seaborn as sns
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=6)
from keras.models import Model
import ray
from sklearn.model_selection import train_test_split
# Importaciones de TensorFlow
import tensorflow as tf
from tensorflow.keras import Sequential, backend as K
from tensorflow.keras.layers import (Input, Dense, TimeDistributed, LSTM, GRU, Dropout,  
                         Concatenate, Flatten, RepeatVector, Lambda, Bidirectional, SimpleRNN, concatenate)
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import (Callback, TensorBoard, EarlyStopping,
                             ModelCheckpoint, CSVLogger, ProgbarLogger)
from sklearn.preprocessing import StandardScaler
from astroNN.lamost import pseudo_continuum


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


2023-08-28 19:01:36.758694: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-28 19:01:36.781888: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-28 19:01:37.136793: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def sampling(samp_args):
    z_mean, z_log_sigma = samp_args

    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_sigma) * epsilon

def preprocess(X_raw, m_max=np.inf):
    X = X_raw.copy()
    wrong_units =  np.all(np.isnan(X[:, :, 1])) | (np.nanmax(X[:, :, 1], axis=1) > m_max)
    X = X[~wrong_units, :, :]
    X[:, :, 0] = times_to_lags(X[:, :, 0])
    return X

#functions taken from https://github.com/yutarotachibana/CatalinaQSO_AutoEncoder

def times_to_lags(T):
    """(N x n_step) matrix of times -> (N x n_step) matrix of lags.
    First time is assumed to be zero.
    """
    assert T.ndim == 2, "T must be an (N x n_step) matrix"
    return np.c_[np.diff(T, axis=1)/365., np.zeros(T.shape[0])]

def plot100reconstrucciones(df,titulo):
    # Crear subplots 10x10
    fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(40,40))
    # Aplanar arreglo de subplots
    axes = axes.flatten()
    # Iterar sobre cada subplot y plotear los datos correspondientes
    for i, ax in enumerate(axes):
        index = df.index[i]
        df_star = reconstruccion(decoding,lcs_scaled_W,lcs_raw_W,decoding_10,index)
        error,excess_var,nombre = df.iloc[i][["log_10(R)","Excess_var","SPICY"]]
        ax.errorbar(df_star.mjd, df_star.mag,df_star.mag_err, label='Original Lc',color="black",marker=".", linestyle = 'dashed')
        ax.errorbar(df_star.mjd,df_star.rec_mag,df_star.rec_mag_std, label='Reconstructed Lc',color="red",marker=".", linestyle = 'dashed')
        ax.fill_between(df_star.mjd,df_star.rec_mag_min,df_star.rec_mag_max, alpha=0.2,facecolor='#D42F4B')
        ax.invert_yaxis()
        ax.set_title(f"SPICY: {nombre}\nlog(R): {error:.4f},\nE_var: {excess_var:.2e}")
        
        # Ajustar ticks en el eje y
        yticks = np.linspace(df_star.mag.min(), df_star.mag.max(), 4)
        yticks_formatted = [round(ytick, 2) for ytick in yticks]
        ax.set_yticks(yticks_formatted)
        
    plt.tight_layout()
    plt.savefig(f"{titulo}.pdf",bbox_inches='tight')
    plt.show()
    
def norm_espectro(df):
    # Datos originales
    # spectra_errs refers to the inverse variance array provided by LAMOST
    # spectra can be multiple spectra at a time
    norm_spec, norm_spec_err = pseudo_continuum(df["flux"].values,
                                            df["ivar"].values,
                                            df["londa"].values, dr=5)
    df["flux_norm"] = norm_spec
    df["flux_error"] = norm_spec_err
    return df
    
@ray.remote(num_returns=2)
def abrir_fits(spec_name,directory):
# Ruta al archivo FITS
    ruta_archivo = spec_name

    # Abrir el archivo FITS
    hdul = fits.open(f"{directory}/{ruta_archivo}")

    # Acceder al header primario (HDU 0)
    header = hdul[0].header

    # Acceder a los datos (HDU 1)
    data = hdul[1].data
    a = np.array(hdul[1].data)
    df = pd.DataFrame(a[0][0],columns=["flux"])
    df["ivar"] = a[0][1]
    df["londa"] = a[0][2]
    df["andmask"] = a[0][3]
    df["ormask"] = a[0][4]
    df["normalization"] = a[0][5]
    df = df.astype(float)
    df = df.loc[(df["ivar"]!=0)]
    hdul.close()
    df = norm_espectro(df)
    df = df.loc[(df["londa"]>3900)&(df["londa"]<5100)].reset_index(drop=True)
    df = df.sort_values(by="londa",ascending=True)
    return header["OBSID"],df[["londa","flux_norm"]].values

In [3]:
catalog = pd.read_csv("zari_Lamost_skiff_test.csv")

In [4]:
spectros_descargados = pd.DataFrame(glob.glob("descarga_lamost/*"),columns=["name"])
spectros_descargados["name"] = spectros_descargados["name"].str.split("/").str[-1]

In [5]:
directory = "descarga_lamost"
spec = []
obsid = []
for espectros in spectros_descargados["name"]:
    result_obid,result_spec = abrir_fits.remote(espectros,directory)
    spec.append(result_spec)
    obsid.append(result_obid)

2023-08-28 19:01:41,119	INFO worker.py:1528 -- Started a local Ray instance.


In [6]:
result_espectros = ray.get(spec)
result_obid = ray.get(obsid)

In [7]:
spectros_descargados["obsid"] = result_obid

In [8]:
spectros_descargados = spectros_descargados.merge(catalog,how="inner",on="obsid")

In [9]:
espectros_padd = pad_sequences(result_espectros, value=np.nan, dtype='float', padding='post')


In [10]:
lcs_scaled  = preprocess(espectros_padd)

In [11]:
lcs_scaled[np.isnan(lcs_scaled)] = 0.

In [12]:
lr = 1e-3 #learning rate 
optimizer = tf.keras.optimizers.legacy.Adam(lr=lr)
output_size=16
gru_size = 32
nepochs = 2000
batchsize = 512
dropout_val = 0.25

resume_training = False

/home/variablestars/miniconda3/envs/deep/lib/python3.8/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
features = ['j_m', 'h_m', 'ks_m','parallax','pmdec','pmra','phot_bp_mean_mag','phot_rp_n_obs','phot_g_mean_mag']

In [14]:
scaler = StandardScaler()
for feature in features:
    spectros_descargados[feature] = scaler.fit_transform(spectros_descargados[[feature]])

In [15]:
idx_train,idx_test = train_test_split(spectros_descargados.index)

In [16]:
X_train = lcs_scaled[idx_train]
X_test = lcs_scaled[idx_test]
params_train = spectros_descargados[features].values[idx_train]
params_test = spectros_descargados[features].values[idx_test]

In [17]:
# Definición de las entradas
main_input = Input(shape=(X_train.shape[1], 2), name='main_input') #(lag, mag)
param_input = Input(shape=(9,), name='param_input') # Entrada para los parámetros

#encoder
encoder_input = concatenate([main_input, RepeatVector(X_train.shape[1])(param_input)])
encoder = Bidirectional(GRU(gru_size, name='encoder1', return_sequences=True))(encoder_input)
encoder = Dropout(dropout_val, name='drop_encoder1')(encoder) 
encoder = Bidirectional(GRU(gru_size, name='encoder2', return_sequences=False))(encoder)
encoder = Dropout(dropout_val, name='drop_encoder2')(encoder)
codings_mean = Dense(units=output_size, name='encoding_mean', activation='linear')(encoder)
codings_log_var = Dense(units=output_size, name='encoding_log_var', activation='linear')(encoder) 
codings = Lambda(sampling, output_shape=(output_size,))([codings_mean, codings_log_var])

#decoder
decoder = RepeatVector(X_train.shape[1], name='repeat')(codings)
decoder = GRU(gru_size, name='decoder1', return_sequences=True)(decoder)
decoder = Dropout(dropout_val, name='drop_decoder1')(decoder)
decoder = GRU(gru_size, name='decoder2', return_sequences=True)(decoder)
decoder = TimeDistributed(Dense(1, activation='linear'), name='time_dist')(decoder)

#VAE
model = Model([main_input, param_input], decoder)

latent_loss = -0.5*K.sum(1+codings_log_var-K.exp(codings_log_var)-K.square(codings_mean),axis=-1)
model.add_loss(K.mean(latent_loss)/200.)  
model.compile(optimizer=optimizer, loss='mse',  metrics=[tf.keras.metrics.MeanAbsoluteError()], weighted_metrics=[tf.keras.metrics.MeanAbsoluteError()], sample_weight_mode='temporal')



2023-08-28 19:03:01.340416: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-28 19:03:01.340439: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: Cacique
2023-08-28 19:03:01.340445: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: Cacique
2023-08-28 19:03:01.340551: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 470.182.3
2023-08-28 19:03:01.340564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 470.182.3
2023-08-28 19:03:01.340567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 470.182.3
2023-08-28 19:03:01.447274: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start abort

In [18]:
model.fit([X_train, params_train],[X_train, params_train], epochs=10,
          validation_split=0.2, batch_size=32)

Epoch 1/10


2023-08-28 19:03:02.081992: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-28 19:03:02.082733: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-28 19:03:02.083242: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-08-28 19:03:03.186070: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-28 19:03:03.186828: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-28 19:03:03.187346: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-08-28 19:03:04.260627: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]


12/54 [=====>........................] - ETA: 29s - loss: 9.3385 - mean_absolute_error: 0.7599 - weighted_mean_absolute_error: 0.7599 

KeyboardInterrupt: 